Overwriting aoai.env


In [10]:
!pip install --upgrade openai
!pip install --upgrade load_dotenv os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [1]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os 
load_dotenv('aoai.env')
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")

In [2]:
import json 
import requests
import time
from openai import AzureOpenAI
client=AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview"
)

In [3]:
load_dotenv('aoai.env')

api_key = os.getenv("OPENWEATHERMAP_API_KEY") 
api_key

'ddff3937d1ab0f0f5722e3d0ed5eef8b'

In [4]:
#def function for get weather

import requests

def get_weather(country) -> dict:
    """
    Fetch weather from OpenWeatherMap using latitude and longitude.

    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        units (str): Units ('standard', 'metric', or 'imperial').

    Returns:
        dict: Weather summary.
    """
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")  # Replace with your actual key

    params = {
        'country': country,
        'appid': api_key
    }
    url = f"https://api.openweathermap.org/data/2.5/weather?q={params['country']}&appid={params['appid']}"


    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            
            "weather_condition": data['weather'][0]['description'],
            "temp": data['main']['feels_like'],
            
        }
    else:
        return {
            "error": f"Weather request failed: {response.status_code} - {response.text}"
        }


In [5]:
get_weather('pakistan')

{'weather_condition': 'overcast clouds', 'temp': 299.44}

In [6]:
 #Define the tools list with the functions the assistant can use
tools_list = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the weather condition using country name. If any argument is missing in the user message, assume it as 'null' and not zero (0) or don't return missing argument value yourself",
        "parameters": {
            "type": "object",
            "properties": {
                "country": {
                    "type": "string",
                    "description": "Name of country or city"
                }
            },
            "required": ["country"]
        }
    }
}]


In [7]:
assistant=client.beta.assistants.create(
    name='weather assistant',
    instructions='you are a weather bot that provides weather information in real time',
    tools=tools_list,
    model='o3-mini'
)
thread=client.beta.threads.create() #create conversation thread

/tmp/ipykernel_1413/3564978942.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread=client.beta.threads.create() #create conversation thread


In [8]:
import time
import json

def run_conversation(user_message):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    while True:
        time.sleep(5)
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

        if run_status.status == "completed":
            messages = client.beta.threads.messages.list(
                thread_id=thread.id
            ).data

            last_response = next((msg for msg in messages if msg.role == "assistant"), None)
            if last_response:
                assistant_message_content = last_response.content[0].text.value
                print(assistant_message_content)
            break

        elif run_status.status == "requires_action":
            required_actions = run_status.required_action.submit_tool_outputs.model_dump()
            tool_outputs = []

            for action in required_actions['tool_calls']:
                func_name = action['function']['name']
                arguments = json.loads(action['function']['arguments'])

                if func_name == "get_weather":
                    output = get_weather(country=arguments.get('country'))
                    tool_outputs.append({
                        "tool_call_id": action['id'],
                        "output": json.dumps(output)
                    })
                else:
                    raise ValueError(f"Unknown function: {func_name}")

            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
        else:
            print("Waiting for assistant to process...")


In [ ]:
while True:
    user_message=input("Enter your message and type end to stop")
    if user_message.lower()=='end':
        print("conversation ended")
        break
    run_conversation(user_message)

Enter your message and type end to stop get me weather for toronto


/tmp/ipykernel_1413/538792259.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(
/tmp/ipykernel_1413/538792259.py:11: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/tmp/ipykernel_1413/538792259.py:18: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(
/tmp/ipykernel_1413/538792259.py:51: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.runs.submit_tool_outputs(
/tmp/ipykernel_1413/538792259.py:24: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


Currently in Toronto, it's overcast with clouds and the temperature is around 293.97 Kelvin, which is roughly 21°C. Enjoy your day outdoors if you plan to go out!


Enter your message and type end to stop Nice thanks


You're welcome! If you need any more weather updates or additional information, just let me know.


Enter your message and type end to stop Get me whether for New Delhi


Currently in New Delhi, it's experiencing haze conditions. The temperature is around 309.83 Kelvin, which is roughly 36.68°C. Stay safe and keep hydrated if you're heading out!
